# Monitoring Environmental Waste Utilization Scores
# A guide for generating EWU-Dashboards
**Sarah Schmidt & David Laner**  
*January 2023*

## 1. Configuration

### 1.1 Import packages

In [3]:
import pandas as pd
import numpy as np
import brightway2 as bw
import premise
from mycolorpy import colorlist as mcp
import string

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [4]:
# create a list with MS Excel column indices (A, B, ..., Z, AA, AB, ...)
# will be used for preparing calculations in MS Excel
alphabet = list(string.ascii_uppercase)
excel_cols=[]

for i in range(100):
    n=0
    j=i
    while j-len(alphabet)>=0:
        j=j-len(alphabet)
        n=n+1
    if n>0:
        col=alphabet[n-1]+alphabet[j]
    else:
        col=alphabet[j]
    excel_cols.append(col)

In [8]:
conversion_factors={'kg':1,
                    't':1e3,
                    'kt':1e6,
                    'g':1e-3,
                    'Mg':1e3,
                    'Gg':1e6}

### 1.2 Gather case study-specific information

In [19]:
# read various case study specific information from input data file (ExcelTool_GeneratorInput_Template.xlsx)
general_info=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='GeneralInformation', index_col=0)
general_info=general_info[general_info.columns[0]].to_dict()

Pathway_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
Pathway_codes=Pathway_codes[Pathway_codes['Pathway tag'].notnull()]['Activity code'].to_list()
Pathway_names=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
Pathway_names=Pathway_names[Pathway_names['Pathway tag'].notnull()]['Pathway tag'].to_list()

WasteGen_code=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
WasteGen_code=WasteGen_code[WasteGen_code['Waste generation'].notnull()]['Activity code'].iloc[0]

act_codes=[WasteGen_code,*Pathway_codes]
act_names=[general_info['Acronym'],*Pathway_names]

# materials for calculation of the environmental impact of materials
material_names=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='EnvironmentalValue')
material_names=material_names['Material Name'].to_list()
material_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='EnvironmentalValue')
material_codes=material_codes['Material Code'].to_list()

waste_utilization_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
waste_utilization_codes=waste_utilization_codes[pd.isnull(waste_utilization_codes['Waste utilization'])==False]['Activity code'].to_list()

# activity tags for contribution analysis
activity_tags=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
activity_tags=activity_tags['Activity tag'].to_list()
activity_tags=[tag for tag in activity_tags if pd.isnull(tag)==False]
activity_tags=list(set(activity_tags))
activity_tags.append('Others')

# waste quantity in kg
WQ_kg=general_info['Waste quantity']*conversion_factors[general_info['Unit']]

### 1.3 Setup and import of databases

#### 1.3.1 Setup

In [11]:
# create a new project or open an existing project
bw.projects.set_current(general_info['Project name'])

In [12]:
# creates the database "biosphere 3"
bw.bw2setup() 
biosphere = bw.Database("biosphere3")

Biosphere database already present!!! No setup is needed


#### 1.3.2 Ecoinvent

In [13]:
# import of the ecoinvent database
db_default_name=general_info['Database name']+'_default'
if db_default_name in bw.databases:
    print("Database has already been imported.")
    eidb_default = bw.Database(db_default_name)
else:
    # mind that the ecoinvent file must be unzipped; then: path to the datasets subfolder
    fpeidbcut = r"{}".format(general_info['Database file path'])
    # the "r" makes sure that the path is read as a string - especially useful when you have spaces in your string
    eidbcut = bw.SingleOuPathwayutEcospold2Importer(fpeidbcut, general_info['Database name']+'_default')
    eidbcut
    eidbcut.apply_strategies()
    eidbcut.statistics()
    eidb_default=eidbcut.write_database()

Database has already been imported.


In [14]:
# copy of the unmodified version of the ecoinvent database
if general_info['Database name'] in bw.databases:
    print("Database has already been imported.")
else:
    eidb_default.copy(general_info['Database name'])
eidb = bw.Database(general_info['Database name'])

Database has already been imported.


#### 1.3.3 Prospective Databases (premise)

In [15]:
# gather information which prospective scenarios shall be created
premise_scenarios=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                                skiprows=3, nrows=8)
premise_update=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                                index_col=0, skiprows=15, nrows=9, usecols='A:B')
premise_update=premise_update[premise_update['Update']=='yes']

In [17]:
# read encryption key 
# (to be requested from the premise library maintainers if you want ot use default scenarios included in `premise`)
encryption_key=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                         usecols = "B", header = 0, nrows=0).columns[0]

In [21]:
if general_info['Generate background system scenarios']=='yes':
    if general_info ['Type of background system scenarios']=='premise':
        
        premise_scenarios_dictlist=[]
        premise_scenario_names_list=[]

        for i in premise_scenarios.index:
            for c in premise_scenarios.columns[3:]:
                if premise_scenarios.loc[i,c]=='x':
                    scenario_dict={"model":premise_scenarios.loc[i,'IAM'], 
                                   "pathway":premise_scenarios.loc[i,'SSP']+'-'+premise_scenarios.loc[i,'RCP'], 
                                   "year":c}
                    scenario_name=premise_scenarios.loc[i,'IAM']+str(c)+'_'+premise_scenarios.loc[i,'SSP']+'-'+premise_scenarios.loc[i,'RCP']
                    premise_scenarios_dictlist.append(scenario_dict)
                    premise_scenario_names_list.append(scenario_name)
       
                    if scenario_name not in bw.databases:
                        ndb = premise.NewDatabase(
                            scenarios=[scenario_name],
                            source_db=eidb_default.name, # name of the database in the BW2 project. Must be a string.
                            source_version=general_info['Database version'], # version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
                            key=encryption_key,# <-- decryption key
                            quiet=True
                            # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
                            )


                        if len(premise_update)==8:
                            ndb.update_all()
                        else:
                            if 'Electricity' in premise_update.index:
                                ndb.update_electricity()
                            if 'Cement' in premise_update.index:
                                ndb.update_cement()
                            if 'Steel' in premise_update.index:
                                ndb.update_steel()
                            if 'Fuels' in premise_update.index:
                                ndb.update_fuels()
                            if 'Cars' in premise_update.index:
                                ndb.update_cars()
                            if 'Trucks' in premise_update.index:
                                ndb.update_trucks()
                            if 'Two wheelers' in premise_update.index:
                                ndb.update_two_wheelers()
                            if 'Buses' in premise_update.index:
                                ndb.update_buses()

                        ndb.write_db_to_brightway(name=[scenario_name])    

#### 1.3.4 Manual Background Scenarios

In [22]:
background_scenarios=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', 
                                   sheet_name='BackgroundScenarios')
background_scenarios

,background scenario,database,input,input code,input unit,input location / biosphere category,activity,activity code,activity location,amount,type


In [23]:
if general_info['Generate background system scenarios']=='yes':
    if (general_info ['Type of background system scenarios']=='manual') or (general_info ['Type of background system scenarios']=='manual + premise'):
        
        background_scenario_names_list=[]
        
        for i in background_scenarios.index:
            db_name=background_scenarios.loc[i,'database']+'_'+background_scenarios.loc[i,'background scenario']
            if db_name not in bw.databases:
                db=bw.Database(background_scenarios.loc[i,'database'])
                db.copy(db_name)
            else:
                print("Database has already been imported.")
            bsdb=bw.Database(db_name)
            
            act=bsdb.get(background_scenarios.loc[i,'activity code'])
            exc=[exc for exc in act.exchanges() if exc.input.as_dict()['code']==background_scenarios.loc[i,'input code']][0]
            
            exc['amount']=background_scenarios.loc[i,'amount']
            exc.save()
            
            if db_name not in background_scenario_names_list:
                background_scenario_names_list.append(db_name)

#### 1.3.5 Overview of databases to be included in the EWU-Dashboard

In [24]:
dbs=[eidb]

db_names_dict={}
db_names_dict['default']=eidb.name
n=1

if general_info['Generate background system scenarios']=='yes':
    if general_info ['Type of background system scenarios']=='premise':
        for scenario in premise_scenario_names_list:
            dbs.append(bw.Database(scenario))
            db='db'+str(n)
            db_names_dict[db]=scenario
            n=n+1

if general_info['Generate background system scenarios']=='yes':
    if (general_info ['Type of background system scenarios']=='manual') or (general_info ['Type of background system scenarios']=='manual + premise'):
        for scenario in background_scenario_names_list:
            dbs.append(bw.Database(scenario))
            db='db'+str(n)
            db_names_dict[db]=scenario
            n=n+1
            
db_names=[db for db in db_names_dict.keys()]